In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.profiler import Profiler, AdvancedProfiler
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl

/usr/workspace/mlmfem/anaconda3/envs/ptl_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path='.'

In [3]:
scaled_Qdata = pd.read_csv(f'{path}/scaled_QdataKn27width50microns.csv', header=0, index_col=0)  #qdata with Knudsen
dropKn = False
if (dropKn):
    for col in scaled_Qdata.columns:
        if (col[:2]=='Kn'):
            scaled_Qdata = scaled_Qdata.drop(col)
            #scaled_Qdata[col] = 1.0
print(scaled_Qdata.shape)
scaled_Qdata.head()

(9735, 138)


,Z_0,Z_1,Z_2,Z_3,Z_4,Z_5,Z_6,Z_7,Z_8,Z_9,...,n_20,n_21,n_22,n_23,n_24,n_25,n_26,Qimpact_c-1,Qimpact_c,Qimpact_c+1
0.012527,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,...,-0.474188,-0.474155,-0.474122,-0.474088,-0.474055,-0.474022,-0.473988,-0.966699,-0.965079,-0.963459
0.012546,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,...,-0.474184,-0.474151,-0.474118,-0.474085,-0.474052,-0.474018,-0.473985,-0.966537,-0.964917,-0.963297
0.012565,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,...,-0.474181,-0.474148,-0.474115,-0.474082,-0.474048,-0.474015,-0.473982,-0.966375,-0.964755,-0.963135
0.012584,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,...,-0.474178,-0.474145,-0.474112,-0.474078,-0.474045,-0.474012,-0.473978,-0.966213,-0.964593,-0.962973
0.012603,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,-0.48868,...,-0.474175,-0.474141,-0.474108,-0.474075,-0.474042,-0.474008,-0.473975,-0.966051,-0.964431,-0.962811


In [4]:
# convert my predicted values back to actual numbers
data_scaling=pd.read_csv(f'{path}/data_scaling.csv', header=0, index_col=0)
Kn_mean = data_scaling['Kn'].loc['mean']; Kn_std = data_scaling['Kn'].loc['std']
Q_mean = data_scaling['Qimpact'].loc['mean']; Q_std = data_scaling['Qimpact'].loc['std']

In [5]:
validationsize = int(0.2*scaled_Qdata.shape[0]) # 20% of samples for validation
testsize = int(0.2*scaled_Qdata.shape[0]) # 20% of samples for test

In [6]:
shuffled_Qdata=scaled_Qdata.sample(n=scaled_Qdata.shape[0])

validation_data=shuffled_Qdata.iloc[:validationsize]
test_data=scaled_Qdata # All data (not randomized) is kept for visualization of model
train_data=shuffled_Qdata.iloc[validationsize+testsize:]

print('Training dataset size: ', train_data.shape, '\nTesting dataset size: ',test_data.shape,'\nValidation dataset size: ', validation_data.shape)

Training dataset size:  (5841, 138) 
Testing dataset size:  (9735, 138) 
Validation dataset size:  (1947, 138)


In [7]:
threePoint = False
if (threePoint):
    target_fields = ['Qimpact_c-1','Qimpact_c','Qimpact_c+1']
else:
    target_fields = ['Qimpact_c']
drop_fields = ['Qimpact_c-1','Qimpact_c','Qimpact_c+1']

train_features, train_targets = train_data.drop(drop_fields, axis=1), train_data[target_fields]
test_features, test_targets = test_data.drop(drop_fields, axis=1), test_data[target_fields]
validation_features, validation_targets = validation_data.drop(drop_fields, axis=1), validation_data[target_fields]

In [8]:
l_rate = 0.05
mse_loss = nn.MSELoss(reduction = 'mean')

In [9]:
Nfeatures = train_features.shape[1]
Nfields = 5 # 4 # if Kn dropped
Npoints = int(Nfeatures / Nfields)
print(f'Number of points per field {Npoints}')

Number of points per field 27


In [10]:
class Regression(pl.LightningModule):
    
### The Model ### 

    # Question: what will your model architecture look like?
    # Initialize the layers
    # Here we have one input layer (size 56 #3N as we have 56 #3N features), one hidden layer (size 10), 
    # and one output layer (size 1 as we are predicting a single value)
    def __init__(self):
        super(Regression, self).__init__()
        N1 = 20
        N2 = 10
        self.fc1 = nn.Linear(Nfeatures, N1)
        self.fc2 = nn.Linear(N1, N2)
        if (threePoint):
            self.fc3 = nn.Linear(N2, 3)
        else:
            self.fc3 = nn.Linear(N2, 1)            

    def forward(self, x):
        #print(f'Is this Kn? {x[0, 3*Npoints:4*Npoints]}')
        # Extract features
        featureName = ['Z', 'T', 'gradT', 'Kn', 'n']
        feature = {}
        i = 0
        for name in featureName:
            feature[name] = x[:, i * Npoints:(i+1) * Npoints]
            mean, std = data_scaling[name][0], data_scaling[name][1]
            feature[name] = x[:, i * Npoints:(i+1) * Npoints] * std + mean
            i = i+1
            #print(f'feature name {name}, shape {feature[name].shape}')
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        # Set output to the local heat flux if no large Kn in the kernel interval
        Kn = feature['Kn']
        ip = int(Npoints / 2)
        for i in range(x.shape[0]):
            KnAvg = sum(Kn[i, :]) / Npoints
            #print(f'i {i}, KnAvg {KnAvg}')
            #if (True): #testing local heat flux otherwise use when Kn over the kernel very small (abs(KnAvg) < 1e-3):
            if (KnAvg < 1e0):
                Z = feature['Z'][i, ip]; T = feature['T'][i, ip]; gradT = feature['gradT'][i, ip]
                kQSH = 6.1e+02 # scaling constant corresponding to the SHICK local heat flux
                Qloc = - kQSH / Z * ((Z + 0.24)/(Z + 4.2)) * T**2.5 * gradT
                x[i] = (Qloc - Q_mean) / Q_std
        return x

### Data loader ### 
    def train_dataloader(self):
        train_dataset = TensorDataset(torch.tensor(train_features.values).float(), torch.tensor(train_targets[target_fields].values).float())
        train_loader = DataLoader(dataset = train_dataset, batch_size = 128)
        return train_loader
        
    def val_dataloader(self):
        validation_dataset = TensorDataset(torch.tensor(validation_features.values).float(),\
                                           torch.tensor(validation_targets[target_fields].values).float())
        validation_loader = DataLoader(dataset = validation_dataset, batch_size = 128)
        return validation_loader
    
    def test_dataloader(self):
        test_dataset = TensorDataset(torch.tensor(test_features.values).float(), torch.tensor(test_targets[target_fields].values).float())
        test_loader = DataLoader(dataset = test_dataset, batch_size = 128)
        return test_loader

### The Optimizer ### 
    def configure_optimizers(self):
        return optim.SGD(self.parameters(), lr=l_rate)

### Training ### 
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = mse_loss(logits, y)
        # Add logging
        logs = {'loss': loss}
        return {'loss': loss, 'log': logs}

### Validation ### 
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = mse_loss(logits, y)
        return {'val_loss': loss}

    # Define validation epoch end
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}

### Testing ###     
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = mse_loss(logits, y)
        correct = torch.sum(logits == y.data)
        
        predictions_pred.append(logits)
        predictions_actual.append(y.data)
        return {'test_loss': loss, 'test_correct': correct, 'logits': logits}
    
    # Define test end
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        logs = {'test_loss': avg_loss}      
        return {'avg_test_loss': avg_loss, 'log': logs, 'progress_bar': logs }

In [ ]:
model = Regression()
trainer = Trainer(max_epochs = 100)  # Option to check if it's working at all: fast_dev_run=True 
                                    # Option to implement early stopping: early_stop_callback=True
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 2.7 K 
1 | fc2  | Linear | 210   
2 | fc3  | Linear | 11    
--------------------------------
2.9 K     Trainable params
0         Non-trainable params
2.9 K     Total params
0.012     Total estimated model params size (MB)


/usr/workspace/mlmfem/anaconda3/envs/ptl_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/workspace/mlmfem/anaconda3/envs/ptl_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/workspace/mlmfem/anaconda3/envs/ptl_env/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1558: PossibleUserWarning: The number of 

Epoch 0:  74%|███████████████████████        | 46/62 [00:01<00:00, 27.52it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 1:  74%|███████████████████████        | 46/62 [00:01<00:00, 27.45it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 2:  74%|███████████████████████        | 46/62 [00:01<00:00, 27.87it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 3:  74%|███████████████████████        | 46/62 [00:01<00:00, 27.65it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 4:  74%|███████████████████████        | 46/62 [00:01<00:00, 26.29it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 4:  76%|███████████████████████▌       | 47/62 [00:01<00:00, 26.30it/s, loss=76, v_num=74]


Epoch 5:  74%|███████████████████████        | 46/62 [00:01<00:00, 25.59it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 6:  74%|███████████████████████        | 46/62 [00:01<00:00, 27.74it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 7:  74%|███████████████████████        | 46/62 [00:01<00:00, 26.79it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 8:  74%|███████████████████████        | 46/62 [00:01<00:00, 25.63it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 8:  82%|█████████████████████████▌     | 51/62 [00:01<00:00, 25.99it/s, loss=76, v_num=74]


Epoch 9:  74%|███████████████████████        | 46/62 [00:01<00:00, 27.64it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 10:  74%|██████████████████████▎       | 46/62 [00:01<00:00, 27.76it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 11:  74%|██████████████████████▎       | 46/62 [00:01<00:00, 28.23it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 12:  74%|██████████████████████▎       | 46/62 [00:01<00:00, 28.45it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 12:  89%|██████████████████████████▌   | 55/62 [00:01<00:00, 28.92it/s, loss=76, v_num=74]


...

Epoch 26:  74%|██████████████████████▎       | 46/62 [00:01<00:00, 24.81it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 27:  74%|██████████████████████▎       | 46/62 [00:02<00:00, 20.20it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 28:  74%|██████████████████████▎       | 46/62 [00:02<00:00, 20.35it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 29:  74%|██████████████████████▎       | 46/62 [00:01<00:00, 27.04it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 29:  84%|█████████████████████████▏    | 52/62 [00:01<00:00, 27.15it/s, loss=76, v_num=74]


Epoch 30:  74%|██████████████████████▎       | 46/62 [00:01<00:00, 28.35it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 31:  74%|██████████████████████▎       | 46/62 [00:01<00:00, 28.27it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 32:  74%|██████████████████████▎       | 46/62 [00:01<00:00, 27.80it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 33:  74%|██████████████████████▎       | 46/62 [00:02<00:00, 22.66it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 33:  90%|███████████████████████████   | 56/62 [00:02<00:00, 22.56it/s, loss=76, v_num=74]


Epoch 34:  74%|██████████████████████▎       | 46/62 [00:01<00:00, 27.39it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 35:  74%|██████████████████████▎       | 46/62 [00:01<00:00, 26.34it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 36:  74%|██████████████████████▎       | 46/62 [00:01<00:00, 28.10it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 37:  74%|██████████████████████▎       | 46/62 [00:02<00:00, 20.21it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 37:  97%|█████████████████████████████ | 60/62 [00:02<00:00, 20.80it/s, loss=76, v_num=74]


Epoch 38:  74%|██████████████████████▎       | 46/62 [00:02<00:00, 20.55it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 39:  74%|██████████████████████▎       | 46/62 [00:02<00:00, 20.63it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 40:  74%|██████████████████████▎       | 46/62 [00:02<00:00, 20.35it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 41:  74%|██████████████████████▎       | 46/62 [00:02<00:00, 20.42it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 42:  74%|██████████████████████▎       | 46/62 [00:02<00:00, 20.60it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Validation:   0%|                                                        | 0/16 [00:00<?, ?it/s]


Epoch 43:  74%|██████████████████████▎       | 46/62 [00:02<00:00, 20.50it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 44:  74%|██████████████████████▎       | 46/62 [00:01<00:00, 27.71it/s, loss=76, v_num=74]
Validation: 0it [00:00, ?it/s]
Epoch 45:   5%|█▍                           | 3/62 [00:00<00:01, 30.30it/s, loss=72.5, v_num=74]

In [ ]:
predictions_pred = []
predictions_actual = []
trainer.test()
predictions_pred

In [ ]:
torch.save(model.state_dict(), f'{path}/model.pt')
model.eval()

In [ ]:
print('length of pred_actual',len(predictions_actual),'\nsize of tenzors inside pred_actual',predictions_actual[3].size())

In [ ]:
# convert my predicted values back to actual numbers
data_scaling=pd.read_csv(f'{path}/data_scaling.csv', header=0, index_col=0)
mean = data_scaling['Qimpact'][0]
std = data_scaling['Qimpact'][1]

# Get predicted points (scaled back to their original size)
if (threePoint):
    Qcomponent = 1
else:
    Qcomponent = 0
plot_pred = []
for i in range(len(predictions_pred)):
    plot_pred.extend(predictions_pred[i].T.numpy()[Qcomponent] * std + mean)
# Save the results to the text file
np.savetxt('Qimpact-NN.txt', np.transpose([scaled_Qdata.index, plot_pred]))

# Get actual points (scaled back to their original size)
plot_actual = []
for i in range(len(predictions_actual)):
    plot_actual.extend(predictions_actual[i].T.numpy()[Qcomponent] * std + mean) 

In [ ]:
# And finally we can see that our network has done a decent job of estimating!
fig1, ax1 = plt.subplots(figsize=(10,4))
ax1.plot(scaled_Qdata.index,plot_actual, 'r', label='Data')
ax1.plot(scaled_Qdata.index,plot_pred, 'k--', label='Prediction')
ax1.set_xlabel('cm')
ax1.set_ylabel('W/cm$^2$')
ax1.legend()

In [ ]:
gradQpred=np.gradient(plot_pred, scaled_Qdata.index)
gradQact=np.gradient(plot_actual, scaled_Qdata.index)

In [ ]:
fig2, ax2 = plt.subplots(figsize=(10,4))
ax2.plot(scaled_Qdata.index,gradQpred, label='Prediction grad')
ax2.plot(scaled_Qdata.index,gradQact, label='Data grad', color='orange')
ax2.legend()
ax2.set_xlabel('cm')
ax2.set_ylabel('W/cm$^3$')